In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV


#### Imports for Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

## Gradient Boosting

* History
    - Leo Breiman (1998,1999) formulated ADABoost as gradient descent with special loss function
    - Friedman (2000,2001) formulated ADABoost with generalized  loss function
* Formulates the boosting problem as an optimization problem using a Loss function
    - Gradually reduce the loss by adding more weak learners
* For regression, fit each weak learner to the residuals of the previous weak learner
* Can be used for regression and classification
    - Classification is more difficult

### Compare with ADABoost

#### Similarities

* Like ADABoost we iteratively add weak learners to build a stronger learner to improve performance

#### Differences

* Gradient Boosting identifies errors by residuals; ADABoost identifies errors by high weight values, 
* Gradient Boosting scales trees by constant amount, the learning rate, versus voting power $\alpha$s in ADABoost
    - Although sklearn does have a learning rate parameter for AdaBoost??
* Gradient Boosting grows more complex trees (i.e. trees with 8 to 32 leaves), ADABoost almost always uses stumps



### Core Ideas

Given: $(x_1,y_1),(x_2,y_2),...,(x_n,y_n)$

Problem: Fit a model F(x) to minimize square loss

Think of a tree as a function, inputs a new value and outputs a prediction for that input (derived from the leaf node)

Initial model F(x) is good but not perfect, for example, its Mean Squared Error is 20.


You can add a model h(x) to F(x): so the new prediction will be F(x) + h(x)

Ideally we would like h(x) to satisfy:

$$F(x_1) + h(x_1) = y_1$$
$$F(x_2) + h(x_2) = y_2$$
$$\vdots$$
$$F(x_n) + h(x_n) = y_n$$

Equivalently:

$$h(x_1) = y_1 - F(x_1) $$
$$h(x_2) = y_2 - F(x_2) $$
$$\vdots$$
$$h(x_n) = y_n - F(x_n) $$

$y_i - F(x_i)$ are the residuals.

By fitting h(x) to the residuals we will get a better model since the residuals indicate which data points are problematic.


Fit h(x) to:
$$ (x_1,y_1 - F(x_1)),(x_2,y_2 - F(x_2)),...,(x_n,y_1 - F(x_n))$$

We can repeat this process to keep improving the fit, so 

$$ F(x) + \gamma h_1(x) + \gamma h_2(x) + ... + \gamma h_m(x)$$

### The Gradient in Gradient Boosting

#### Gradient Descent

Minimize a loss function L by moving in the opposite direction of the gradient.

$$ \theta_{t + 1} = \theta_t - \gamma \frac{\partial{L}}{\partial{\theta_t}}$$

In [ ]:
x = np.linspace(-2,2,100)
y = x**2
plt.plot(x,y)
dy = 2*x
x_1 = 1
y_1 = x_1**2
plt.plot(x_1,y_1,'r>',MarkerSize = 10)
z = np.linspace(0,2)
y_line = 2*(z - x_1) + y_1
plt.plot(z,y_line,'b-')
plt.title("Positive slope, move down the curve  ");

Loss function: $L(y, F(x)) = \frac{1}{2}(y - F(x))^2$
    (Note: this 1/2 times OLS Loss Function (i.e. RSS))

Minimize $ J = \sum_i{L(y_i, F(x_i))}$ by adjusting $F(x_1),F(x_2), \ldots, F(x_n)$

Take derivatives: 

$$\frac{\partial{J}}{\partial{F(x_i)}} = \frac{\partial{\sum_i L(y_i,F(x_i))}}{\partial{F(x_i)}} = \frac{\partial{L(y_i,F(x_i))}}{\partial{F(x_i)}} = -(y_i - F(x_i)) = F(x_i) - y_i$$

Therefore the residuals are the negative gradients:

$$ y_i - F(x_i) = - \frac{\partial{J}}{\partial{F(x_i)}}$$

Updating the function is like doing Gradient Descent:

$$ F_{t+1}(x_i) = F_t(x_i) + h(x_i)$$
$$ F_{t+1}(x_i) = F_t(x_i) + y_i - F_t(x_i)$$
$$ F_{t+1}(x_i) = F_t(x_i) - 1\frac{\partial{J}}{\partial{F(x_i)}}$$

For regression with square loss: $residual \Longleftrightarrow negative \text{ }gradient$

Updating by fitting the residual is updating by Gradient Descent

#### Updating by Gradient Descent generalizes to other loss functions

* Square error is not robust to outliers (because of magnifying the error by squaring it)

* Absolute loss L(y,F(x)) = |y - F(x)| is more robust to outliers


In [ ]:
# y - the target
# yhat - the fitted values

def calc_loss(y,yhat):
    return np.mean(1/2*(y - yhat)**2)

def gradient(y,yhat):
    return y - yhat


In [ ]:
Years = [5,7,12,23,25,28,29,34,35,40]
Salary = [82,80,103,118,172,127,204,189,99,166]
df = pd.DataFrame({'Years':Years,'Salary': Salary})
X = df.loc[:,['Years']].values
y = df.loc[:,'Salary'].values
plt.plot(X,y,'o')
plt.xlabel('Years')
plt.ylabel('Salary');

In [ ]:
np.random.seed(1234)

model = DecisionTreeRegressor(max_depth = 3,random_state=42)
yhat = np.mean(y)
loss = []
alpha = .1
u = gradient(y,yhat)
M = 100
for i in range(M):
    model.fit(X,u)
    yhat = yhat + alpha*model.predict(X)
    u = gradient(y,yhat)
    loss.append(calc_loss(y,yhat))
    if i%20 == 0: print(f'Iteration {i}, loss, {loss[-1]}')
    if loss[-1] < .05: 
        print(f'Number of Iterations {i}')
        break


print(f'Initial loss {np.round(loss[0],2)}, Final Loss {np.round(loss[-1],2)}')
print(f'Min residual {np.round(np.min(u),2)}, Max residual {np.round(np.max(u),2)}')
  

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2)
ax1.plot(X,yhat,'o')
ax1.set_ylabel('yhat')
ax1.set_xlabel('Years')
ax2.plot(X,u,'o')
ax2.set_ylim(-10,10)
ax2.set_ylabel('residual')
ax2.set_xlabel('Years')
plt.tight_layout()

In [ ]:
plt.plot(np.arange(i+1),loss)
plt.xlabel('Iteration')
plt.ylabel('Loss');

### Boosting Algorithm for regression trees

* Fit each model to the residuals of the previous model
 
#### Input: $(x_i,y_i)$ for i = 1,2,..,n and a Loss function $L(y_i,F(x))$
* Loss function: $L(y, F(x)) = \frac{1}{2}(y - F(x))^2$
* F(x) is the predicted value, i.e. output of the tree function

#### 1. Initialize model with

<div style="font-size: 115%;">
$$F_0(x) = \underset{\gamma}{\mathrm{argmin}} \sum_i^nL(y_i,\gamma)$$  
</div>

* Initial prediction: average of $y_i$s (the observed values)

#### 2. For m = 1,...,M, i = 1,..,n
* m = tree number, (e.g. 100)
* i = observation number
* Sequentially add trees

#### 2a. Compute psuedo-residuals

<div style="font-size: 115%;">
$$r_{im} = - \frac{\partial L(y_i,F(x_i))}{\partial F(x_i)}$$ 
</div>

* This is just the residual: (observed - predicted) where observed = $y_i$ and predicted = $F_{m-1}(x)$ from last round
* $r_{im}$ is called a pseudo-residual
* Pseudo because of the 1/2

#### 2b. Fit a regression tree to the residuals $r_{im}$s

* The leaf nodes of the tree are regions $R_{jm}$, j = 1,..,$J_m$, the number of leaf nodes

#### 2c. Calculate output values of tree

**For j = 1,...,$J_m$**

<div style="font-size: 115%;">
$$\gamma_{jm} = \underset{\gamma}{\mathrm{argmin}} \sum_{x_i \in R_{ij}}L(y_i,F_{m-1}(x_i) + \gamma)$$
    
* $\gamma_{jm}$ is the output of leaf node j of tree m
* For this Loss function it is just the average of the leaf node values

#### 2d. Update Classifier, make new prediction

* Add output of tree m

<div style="font-size: 115%;">
$$F_m(x) = F_{m - 1}(x) + \nu\sum_{j=1}^{J_m}\gamma_{jm}I(x \in R_{jm})$$
</div>

* $\nu$ is learning rate
* The summation is over data points in the region

#### 3. New data value

* Compute $y_i$ of new data point using $F_M(x)$

### Slow learning 

* Fitting small trees to the residuals slowly improves $F(x)$ in areas where it does not perform well.
* The shrinkage parameter $\nu$ slows the process down even further.
* In general, statistical learning methods that learn slowly tend to perform well.

### Gradient Boosting in sklearn

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html


#### Gradient Boosting Tuning Parameters

* Loss: (default 'ls'- least squares)
    - Loss function L
* n_estimators (default 100) M
    - The number of trees   
* learning_rate (default 0.1) $\nu$: 
    - Shrinkage parameter controls rate of learning
* max_depth: 
    - Control size by limiting tree depth
* min_samples_split: 
    - Minimum # of samples required to split an internal node
* min_samples_leaf: 
    - Control size by setting minimum number of samples at leaf nodes

#### Motorcycle data

https://www.stat.cmu.edu/~larry/all-of-statistics/=data/motor.dat

In [ ]:
mc = pd.read_csv("Motorcycle.csv")
plt.plot(mc.times,mc.accel,'o')
plt.xlabel('times')
plt.ylabel('accel')
mc.head()

In [ ]:
X = mc.loc[:,['times']].values
y = mc.loc[:,'accel'].values
X.shape,y.shape

In [ ]:
model = GradientBoostingRegressor(max_depth = 2, 
                                  criterion = 'mse',
                                  n_estimators = 100,
                                  learning_rate = .1,
                                  subsample = 1,
                                  min_samples_leaf = 1,
                                  random_state=42)
model.fit(X,y)
yhat = model.predict(X)

print('Features: ',model.feature_importances_)
print('R-squared Score: ',np.round(model.score(X,y),3))
print('MSE: ',np.round(np.mean((y - yhat)**2),2))

fig,(ax1,ax2) = plt.subplots(1,2)
ax1.plot(X,y,'o')
ax1.plot(X,yhat,'r')
ax2.plot(X,gradient(y,yhat),'o')
ax2.set_ylim(-10,10)
plt.tight_layout()

In [ ]:
model.get_params()

#### Boston Housing Data

In [ ]:
boston = pd.read_csv('Boston.csv')
boston.head()

In [ ]:
feats = boston.columns.tolist()[0:-1] ## ['crim','rm','age'] ##
print(feats)
X = boston.loc[:, feats].values
y = boston.loc[:, 'medv'].values

# Make Validation Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1234)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Fit model to traing data
m = GradientBoostingRegressor(n_estimators = 500, max_depth =6, learning_rate = 0.1)
m.fit(X_train, y_train)

# Predict the test data
yhat = m.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, yhat)
mse = mean_squared_error(y_test,yhat)
print("R2: %.4f" % r2)
print('MSE',np.round(mse,3))


In [ ]:
# compute test set deviance
test_score = np.zeros((500,), dtype=np.float64)

for i, yhat in enumerate(m.staged_predict(X_test)):
    test_score[i] = m.loss_(y_test, yhat)

plt.figure(figsize=(12, 6))
plt.subplot(1, 1, 1)
plt.title('Loss')
plt.plot(np.arange(500) + 1, m.train_score_, 'b-',label='Training Set Loss')
plt.plot(np.arange(500) + 1, test_score, 'r-',label='Test Set Loss')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:
feature_importance = m.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
feat_names = feats 
sorted_names = [feat_names[i] for i in sorted_idx]
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(12, 6))
plt.subplot(1, 1, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos,sorted_names)
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
model = GradientBoostingRegressor(loss = 'ls', random_state = 42)
parameters = [{'max_depth': [2,4,6,8],
               'n_estimators': [100,200,500],
               'learning_rate': [.01,.1,.5],
               'min_samples_leaf': [1,3],
               'subsample': [.5,1]
               }]
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'r2', #'neg_mean_squared_error',
                           cv = 5,
                           iid = 'False',
                           verbose = 1,
                           n_jobs = -1)
grid_search = grid_search.fit(X, y)
print("Best accuracy: ", grid_search.best_score_)
print("Best parameters: ", grid_search.best_params_ )



### Gradient Boosting Classification

* Same basic algorithm used for regression
* Related to Logistic Regression using the logit (i.e. log odds) and logistic function.
* Predict log(odds) rather than residuals
    - Remember odds = $\frac{p}{1-p}$, p = probability
* Loss function

<div style="font-size: 115%;">
$$ L(y,F(x)) = -ylog(odds) + log(1+e^{log(odds)})$$
</div>

* Derivative of Loss Function
    
<div style="font-size: 115%;">    
$$\frac{\partial L}{F(x)} = -y + logistic(log(odds)) = -y + p$$
</div>

### Gradient Boosting Classification in slkearn


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

#### Gradient Boosting Tuning Parameters

* Loss: (default 'deviance')
    - Logistic Regression for probabilistic outputs
* n_estimators (default 100) M
    - The number of trees
* learning_rate (default 0.1) $\nu$: 
    - Shrinkage parameter controls rate of learning
* max_depth: 
    - Control size by limiting tree depth
* min_samples_split: 
    - Minimum # of samples required to split an internal node
* min_samples_leaf: 
    - Control size by setting minimum number of samples at leaf nodes

In [ ]:
mc.tail()


In [ ]:
sns.scatterplot(x='times',y='accel',hue='strata',data=mc);

In [ ]:
X = mc.loc[:,['times','accel']]
y = mc.loc[:,'strata']
# Make Validation Set
X_train, X_test, y_train, y_test = train_test_split(X, y,  stratify = y,test_size = 0.25, random_state = 1234)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
model = GradientBoostingClassifier(random_state = 42)
model.fit(X_train, y_train)

# Predict the test data
yhat = model.predict(X_test)

cm = confusion_matrix(y_test,yhat)
print(cm)
accuracy_score(y_test,yhat)


### References

**Gradient Boosting Algorithm:** STAT Quest, Gradient Boosting Parts 1 - 4. https://www.youtube.com/watch?v=3CC4N4z3GJc